# 🎯 Pipeline de Vectores STEM para Graduados EPN

**Objetivo**: Generar una matriz de características donde cada fila representa a un individuo mediante un vector consolidado:
```
V_total = [V_blandas (7 dim) + V_técnicas (52 dim)] = 59 dimensiones
```

**Fuentes metodológicas**: O*NET, ESCO, IEEE-SWEBOK

---
## 📦 Fase 0: Setup

In [1]:
import pandas as pd
import numpy as np
import json
import re
import os

# Para quitar acentos
try:
    from unidecode import unidecode
except ImportError:
    !pip install unidecode -q
    from unidecode import unidecode

# Directorio de trabajo
WORK_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f"📁 Directorio de trabajo: {WORK_DIR}")
print("✅ Dependencias listas")

📁 Directorio de trabajo: /home/desarrollo03/Documentos/UNIVERSIDAD/TIC/CARPETA DE TRABAJO
✅ Dependencias listas


---
## 📚 Fase 1: Cargar Diccionario Maestro STEM

52 competencias técnicas con ~400 aliases, basado en O*NET, ESCO, IEEE-SWEBOK.

In [2]:
# Cargar diccionario desde archivo local
with open('diccionario_maestro_stem.json', 'r', encoding='utf-8') as f:
    diccionario_data = json.load(f)

DICCIONARIO_MAESTRO = diccionario_data['competencias']
print(f"✅ Diccionario cargado: {len(DICCIONARIO_MAESTRO)} competencias técnicas")

✅ Diccionario cargado: 52 competencias técnicas


---
## 📊 Fase 2: Cargar Datos de la Encuesta

In [3]:
# Cargar archivo Excel
EXCEL_FILE = "Encuesta recien graduados - pregrado(1).xlsx"
df_raw = pd.read_excel(EXCEL_FILE)
print(f"✅ Archivo cargado: {len(df_raw)} registros, {len(df_raw.columns)} columnas")

✅ Archivo cargado: 380 registros, 42 columnas


---
## 🧠 Fase 3: Extracción de Habilidades Blandas (7 dimensiones)

Las habilidades blandas están en escala Likert 1-5. Se normalizan a [0, 1].

In [5]:
# Definir columnas de habilidades blandas (índices 3-9 del Excel)
HAB_BLANDAS_INDICES = [3, 4, 5, 6, 7, 8, 9]
HAB_BLANDAS_NOMBRES = [
    "hab_gestion",
    "hab_comunicacion", 
    "hab_liderazgo",
    "hab_trabajo_equipo",
    "hab_etica",
    "hab_responsabilidad",
    "hab_aprendizaje"
]

def extract_likert(value):
    """Extrae el valor numérico de una respuesta Likert."""
    if pd.isna(value):
        return np.nan
    value_str = str(value)
    match = re.search(r'^(\d)', value_str)
    if match:
        return int(match.group(1))
    return np.nan

# Extraer habilidades blandas
df_blandas = pd.DataFrame()
for idx, nombre in zip(HAB_BLANDAS_INDICES, HAB_BLANDAS_NOMBRES):
    df_blandas[nombre] = df_raw.iloc[:, idx].apply(extract_likert)

# Normalizar a escala [0, 1]
df_blandas_norm = (df_blandas - 1) / 4

print("=== HABILIDADES BLANDAS (normalizadas 0-1) ===")
print(df_blandas_norm.describe().round(3))

# Verificar NaN
nan_count = df_blandas_norm.isna().sum()
print(f"\n⚠️ Valores faltantes:\n{nan_count}")

=== HABILIDADES BLANDAS (normalizadas 0-1) ===
       hab_gestion  hab_comunicacion  hab_liderazgo  hab_trabajo_equipo  \
count      380.000           380.000        380.000             380.000   
mean         0.680             0.603          0.676               0.709   
std          0.242             0.269          0.255               0.245   
min          0.000             0.000          0.000               0.000   
25%          0.500             0.500          0.500               0.500   
50%          0.750             0.500          0.750               0.750   
75%          0.750             0.750          0.750               1.000   
max          1.000             1.000          1.000               1.000   

       hab_etica  hab_responsabilidad  hab_aprendizaje  
count    380.000              380.000          380.000  
mean       0.784                0.709            0.822  
std        0.230                0.257            0.221  
min        0.000                0.000            

---
## 🔧 Fase 4: Procesamiento de Habilidades Técnicas (52 dimensiones)

Se procesan las respuestas abiertas (columna 35) y se generan vectores binarios.

In [6]:
def match_skills(texto, diccionario):
    """
    Encuentra competencias técnicas en una respuesta de texto.
    
    Args:
        texto: Respuesta abierta del estudiante
        diccionario: Diccionario de competencias técnicas
    
    Returns:
        Lista de skills encontrados
    """
    if pd.isna(texto):
        return []
    
    texto = str(texto).lower()
    texto = unidecode(texto)  # Quitar acentos
    
    matches = []
    for skill_id, skill_info in diccionario.items():
        for alias in skill_info['aliases']:
            alias_clean = unidecode(alias.lower())
            if alias_clean in texto:
                matches.append(skill_id)
                break  # Evitar duplicados del mismo skill
    
    return list(set(matches))

# Procesar columna de respuestas técnicas (índice 35)
COL_TECNICAS = 35
respuestas_tecnicas = df_raw.iloc[:, COL_TECNICAS]

# Generar matches para cada respuesta
matches_por_fila = [match_skills(resp, DICCIONARIO_MAESTRO) for resp in respuestas_tecnicas]

# Crear matriz binaria de habilidades técnicas
skills_list = list(DICCIONARIO_MAESTRO.keys())
df_tecnicas = pd.DataFrame(0, index=range(len(df_raw)), columns=skills_list)

for i, matches in enumerate(matches_por_fila):
    for skill in matches:
        df_tecnicas.loc[i, skill] = 1

# Estadísticas
cobertura = (df_tecnicas.sum(axis=1) > 0).mean()
print(f"✅ Cobertura del diccionario: {cobertura:.1%}")
print(f"\n=== TOP 10 COMPETENCIAS MÁS FRECUENTES ===")
print(df_tecnicas.sum().sort_values(ascending=False).head(10))

✅ Cobertura del diccionario: 96.3%

=== TOP 10 COMPETENCIAS MÁS FRECUENTES ===
analisis_datos             342
inteligencia_artificial    196
sistemas_operativos         82
programacion                73
redes                       64
desarrollo_web              63
hci                         62
gestion_calidad             40
economia_finanzas           39
base_datos                  35
dtype: int64


---
## 🔗 Fase 5: Generación del Vector Consolidado

Se combina `V_blandas (7 dim)` + `V_técnicas (52 dim)` = **V_total (59 dim)**

In [7]:
# Concatenar vectores
V_total = pd.concat([df_blandas_norm, df_tecnicas], axis=1)

print(f"✅ Vector consolidado generado:")
print(f"   - Dimensiones: {V_total.shape}")
print(f"   - Habilidades blandas: {len(HAB_BLANDAS_NOMBRES)} columnas")
print(f"   - Habilidades técnicas: {len(skills_list)} columnas")

# Vista previa
print("\n=== VISTA PREVIA (primeras 5 filas) ===")
V_total.head()

✅ Vector consolidado generado:
   - Dimensiones: (380, 59)
   - Habilidades blandas: 7 columnas
   - Habilidades técnicas: 52 columnas

=== VISTA PREVIA (primeras 5 filas) ===


,hab_gestion,hab_comunicacion,hab_liderazgo,hab_trabajo_equipo,hab_etica,hab_responsabilidad,hab_aprendizaje,programacion,python,java,...,geologia,petroleos,agroindustria,matematica_aplicada,fisica_aplicada,economia_finanzas,gestion_empresarial,auditoria_ti,hci,algoritmos
0,1.00,1.00,1.0,1.00,1.0,1.00,1.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.50,0.50,0.5,0.50,0.5,0.75,1.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.50,0.75,1.0,1.00,1.0,1.00,1.00,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.50,0.75,0.5,0.75,1.0,0.50,0.75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.75,0.50,0.5,0.75,1.0,0.75,0.75,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---
## ✅ Fase 6: Validación y Tests

In [8]:
# === TESTS DE VALIDACIÓN ===

# Test 1: Dimensiones correctas
assert V_total.shape[1] == 7 + 52, "❌ Error: Dimensiones incorrectas"
print("✅ Test 1 PASSED: Dimensiones correctas (59 features)")

# Test 2: Normalización de habilidades blandas
assert df_blandas_norm.min().min() >= 0, "❌ Error: Valores negativos en blandas"
assert df_blandas_norm.max().max() <= 1, "❌ Error: Valores > 1 en blandas"
print("✅ Test 2 PASSED: Normalización correcta [0, 1]")

# Test 3: Valores binarios en técnicas
assert df_tecnicas.isin([0, 1]).all().all(), "❌ Error: Valores no binarios en técnicas"
print("✅ Test 3 PASSED: Valores binarios en técnicas")

# Test 4: Cobertura mínima
cobertura = (df_tecnicas.sum(axis=1) > 0).mean()
assert cobertura >= 0.80, f"❌ Cobertura insuficiente: {cobertura:.1%}"
print(f"✅ Test 4 PASSED: Cobertura {cobertura:.1%} >= 80%")

print("\n🎉 TODOS LOS TESTS PASARON")

✅ Test 1 PASSED: Dimensiones correctas (59 features)
✅ Test 2 PASSED: Normalización correcta [0, 1]
✅ Test 3 PASSED: Valores binarios en técnicas
✅ Test 4 PASSED: Cobertura 96.3% >= 80%

🎉 TODOS LOS TESTS PASARON


---
## 💾 Fase 7: Exportación a CSV

In [9]:
# Agregar columna de carrera para referencia
V_total['carrera'] = df_raw['carrera']

# Guardar localmente
output_csv = "vectores_graduados_stem.csv"
V_total.to_csv(output_csv, index=False)
print(f"✅ Guardado: {output_csv}")

# También como pickle para preservar tipos
output_pkl = "vectores_graduados_stem.pkl"
V_total.to_pickle(output_pkl)
print(f"✅ Pickle guardado: {output_pkl}")

print(f"\n📊 Resumen final:")
print(f"   - Total registros: {len(V_total)}")
print(f"   - Total features: {V_total.shape[1] - 1}")  # -1 por carrera

✅ Guardado: vectores_graduados_stem.csv
✅ Pickle guardado: vectores_graduados_stem.pkl

📊 Resumen final:
   - Total registros: 380
   - Total features: 59
